# Billboard Functions

In [4]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
from glob import glob
from os import getcwd
from os.path import join
from fileUtils import getBasename, getDirname, getBaseFilename
from fsUtils import isFile, isDir, moveFile, removeFile
from webUtils import getHTML, getWebData
from ioUtils import saveJoblib, loadJoblib
import urllib
print("Python: {0}".format(sys.version))

import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-02-14 11:07:08.407337


# Starter Class Section

In [5]:
basedir = getcwd()
try:
    filename = glob(join(basedir, "data", "billboard", "starter.html"))[0]
except:
    print("Could not find starter HTML file!")
fdata = getHTML(filename)

In [6]:
chartData  = {}
dirname = None
baseURL = "https://www.billboard.com"
for iul,ul in enumerate(fdata.findAll("ul")):
    lis = ul.findAll("li", {"class": "chart-group__item"})
    for j,li in enumerate(lis):  
        a = li.find('a')
        if a is not None:
            href = a['href']
            text = a.text
            subdir = getDirname(href)[1:]
            chartData[text] = [baseURL, subdir, getBasename(href)]

# Download Yearly Information

In [7]:
from time import sleep
years = [str(x) for x in range(1958,2014)]
for year in years:
    url="https://www.billboard.com/archive/charts/{0}".format(year)
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,}
    
    savedir = join(basedir, "data", "billboard", "yearly")
    savename = join(savedir, "{0}.p".format(year))
    if isFile(savename):
        continue
    
    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need

    print("Saving {0}".format(savename))
    saveJoblib(data=data, filename=savename, compress=True)
    sleep(1)

# Parse Yearly Information

In [8]:
downloads = {}
baseURL = "https://www.billboard.com"
for ifile in sorted(glob(join(basedir, "data", "billboard", "yearly", "*.p"))):
    year = getBaseFilename(ifile)
    if downloads.get(year) is None:
        downloads[year] = {}
    fdata = getHTML(ifile)
    for iul,ul in enumerate(fdata.findAll("ul")):
        lis = ul.findAll("li", {"class": "chart-group__item"})
        for j,li in enumerate(lis):  
            a = li.find('a')
            if a is not None:
                href = a['href']
                text = a.text
                subdir = getDirname(href)[1:]
                downloads[year][href] = [baseURL, subdir, getBasename(href)]

# Download Category Data

In [10]:
baseURL = "https://www.billboard.com"

for year, yearData in downloads.items():   
    for href, hrefData in yearData.items():
        url  = "{0}/{1}".format(baseURL, href)
        year = getBasename(hrefData[1])
        category = hrefData[2]

        savedir  = join(basedir, "data", "billboard", "categories")
        savename = join(savedir, "{0}-{1}.p".format(year, category))

        if isFile(savename):
            continue

        print("  Trying to download and save {0}".format(savename))
            
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,} 

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need

        print("Saving {0}".format(savename))
        saveJoblib(data=data, filename=savename, compress=True)
        sleep(1)        

  Trying to download and save /Users/tgadfort/Documents/code/charts/data/billboard/categories/2018-billboard-argentina-hot-100.p


HTTPError: HTTP Error 404: Not Found

# Parse Category Data

In [13]:
data = {}

baseURL = "https://www.billboard.com"
names = [getBaseFilename(x) for x in sorted(glob(join(basedir, "data", "billboard", "categories", "*.p")))]
categories = set([x[5:] for x in names])

names = [getBaseFilename(x) for x in sorted(glob(join(basedir, "data", "billboard", "categories", "*.p")))]
years = sorted(set([x[:4] for x in names]))

print("There are {0} years".format(len(years)))
print("There are {0} charts".format(len(categories)))

There are 61 years
There are 90 charts


In [24]:
data = {}
for category in sorted(categories):
    categoryData = {}
    for year in sorted(years):
        for ifile in sorted(glob(join(basedir, "data", "billboard", "categories", "{0}-{1}.p".format(year, category)))):
            try:
                fdata = getHTML(ifile)
            except:
                removeFile(ifile, debug=True)
                continue
            for i,table in enumerate(fdata.findAll('table')):
                ths = table.findAll("th")
                trs = table.findAll("tr")

                headers = [x.text for x in ths]
                for j,tr in enumerate(trs[1:]):
                    tds  = tr.findAll('td')
                    vals = [x.text for x in tds]

                    if len(vals) == 3:
                        date,name,artist = vals
                        try:
                            date = getDateTime(", ".join([date, year]))
                        except:
                            print(vals)
                            1/0
                    elif len(vals) == 1:
                        date = vals[0]
                        try:
                            date = getDateTime(", ".join([date, year]))
                        except:
                            print(vals)
                            1/0

                    if not isDate(date):
                        raise ValueError("Could not form date for {0}".format(date))
                        
                    if data.get(artist) is None:
                        data[artist] = {}
                        print("Artist: {0}".format(artist))
                    if data[artist].get(name) is None:
                        data[artist][name] = {}
                        #print("\tSong: {0} ({1})".format(name, len(data[artist])))
                    if data[artist][name].get(category) is None:
                        data[artist][name][category] = []
                    data[artist][name][category].append([date])

                        
                        
            print("Found {0} artists".format(len(data)))
            
            
for artist,artistData in data.items():
    for name,nameData in artistData.items():
        for category,categoryData in nameData.items():
            data[artist][name][category] = [min(categoryData),max(categoryData)]

Artist: Brook Benton
Artist: Connie Francis
Artist: Joe Dowell
Artist: The Highwaymen
Artist: Bob Moore and His Orch.
Artist: Sue Thompson
Artist: Jimmy Dean
Artist: Henry Mancini
Found 8 artists
Artist: The Lettermen
Artist: Elvis Presley
Artist: Burl Ives
Artist: Kenny Ball and his Jazzmen
Artist: Mr. Acker Bilk
Artist: Ray Charles
Artist: David Rose
Artist: Bobby Vinton
Artist: Nat King Cole
Artist: Frank Ifield
Artist: Gene Pitney
Artist: Brenda Lee
Artist: Steve Lawrence
Found 21 artists
Artist: The Rooftop Singers
Artist: The Cascades
Artist: Skeeter Davis
Artist: Andy Williams
Artist: Peter, Paul & Mary
Artist: Al Martino
Artist: Kyu Sakamoto
Artist: Rolf Harris
Artist: The Village Stompers
Artist: Dale & Grace
Artist: The Singing Nun (Soeur Sourire)
Found 32 artists
Artist: Rick Nelson
Artist: Al Hirt
Artist: Diane Renay
Artist: Louis Armstrong And The All Stars
Artist: The Ray Charles Singers
Artist: Barbra Streisand
Artist: Getz/Gilberto
Artist: Dean Martin
Artist: Gale Garne

Artist: Peabo Bryson & Regina Belle
Artist: Kenny G
Artist: Vanessa Williams/Brian McKnight
Artist: Restless Heart Featuring Warren Hill
Artist: Kenny G With Peabo Bryson
Artist: Tina Turner
Found 331 artists
Artist: John Mellencamp With Me'Shell Ndegeocello
Artist: Sheryl Crow
Artist: Boyz II Men
Found 334 artists
Artist: Melissa Etheridge
Artist: Martin Page
Artist: The Rembrandts
Artist: Seal
Artist: Sophie B. Hawkins
Artist: Mariah Carey & Boyz II Men
Found 340 artists
Artist: Journey
Found 341 artists
Artist: Toni Braxton
Artist: Kenny Loggins
Artist: Jewel
Artist: Bob Carlisle
Artist: Shawn Colvin
Artist: LeAnn Rimes
Found 347 artists
Artist: Savage Garden
Artist: Shania Twain
Artist: Backstreet Boys
Artist: R. Kelly & Celine Dion
Found 351 artists
Artist: Sarah McLachlan
Found 352 artists
Artist: Faith Hill
Artist: Marc Anthony
Artist: Don Henley
Artist: BBMak
Artist: Huey Lewis & Gwyneth Paltrow
Artist: 'N Sync
Found 358 artists
Artist: NewSong
Artist: Lee Ann Womack
Artist: Di

Artist: David Byrne
Artist: Thirty Seconds To Mars
Artist: A Perfect Circle
Artist: Dance Gavin Dance
Artist: Mike Shinoda
Artist: The Story So Far
Found 660 artists
Artist: Siouxsie & The Banshees
Artist: Big Audio
Artist: Psychedelic Furs
Found 663 artists
Artist: Julian Cope
Artist: Lou Reed
Artist: The Replacements
Artist: Elvis Costello
Artist: XTC
Artist: The Cure
Artist: Love And Rockets
Artist: Public Image Ltd.
Artist: The B-52s
Artist: Hoodoo Gurus
Artist: Tears For Fears
Artist: Camper van Beethoven
Artist: Ian McCulloch
Artist: Kate Bush
Artist: The Jesus And Mary Chain
Found 678 artists
Artist: The Psychedelic Furs
Artist: Peter Murphy
Artist: Sinead O'Connor
Artist: Midnight Oil
Artist: The Church
Artist: The Sundays
Artist: World Party
Artist: Concrete Blonde
Artist: Gene Loves Jezebel
Artist: David J
Artist: Jane's Addiction
Artist: The Railway Children
Artist: INXS
Artist: Sisters Of Mercy
Found 692 artists
Artist: Happy Mondays
Artist: Sting
Artist: Jesus Jones
Artist

Found 968 artists
Artist: Michael Jackson
Artist: The Police
Artist: Quiet Riot
Found 971 artists
Artist: Prince And The Revolution
Found 972 artists
Artist: Wham!
Artist: John Fogerty
Artist: Dire Straits
Found 975 artists
Artist: Mr. Mister
Artist: Van Halen
Artist: Patti LaBelle
Found 978 artists
Found 978 artists
Artist: Def Leppard
Artist: Guns N' Roses
Found 980 artists
Artist: Bobby Brown
Artist: Debbie Gibson
Artist: Tone-Loc
Artist: Fine Young Cannibals
Artist: New Kids On The Block
Artist: Milli Vanilli
Artist: Motley Crue
Found 987 artists
Artist: M.C. Hammer
Artist: Vanilla Ice
Found 989 artists
Artist: N.W.A
Artist: Skid Row
Artist: Garth Brooks
Found 992 artists
Artist: Kris Kross
Artist: The Black Crowes
Artist: Billy Ray Cyrus
Artist: Ice Cube
Found 996 artists
Artist: Aerosmith
Artist: Cypress Hill
Artist: Meat Loaf
Artist: Snoop Doggy Dogg
Found 1000 artists
Artist: John Michael Montgomery
Artist: Soundgarden
Artist: Ace Of Base
Artist: Pantera
Artist: Tim McGraw
Foun

Artist: Paul Rodgers
Artist: Dave Alvin & Phil Alvin
Artist: Eric Johnson
Artist: John Hiatt
Artist: Trudy Lynn Featuring Steve Krase
Found 1280 artists
Artist: Steve Earle And The Dukes
Artist: JJ Grey & MOFRO
Artist: Beth Hart
Artist: Samantha Fish
Artist: Bobby Messano
Artist: Billy Gibbons And The BFG's
Found 1286 artists
Artist: The Rides
Artist: Devon Allman
Artist: Mike Zito
Found 1289 artists
Artist: Curtis Knight Featuring Jimi Hendrix
Artist: Robert Cray & Hi Rhythm
Artist: Taj Mahal & Keb' Mo'
Artist: Chuck Berry
Artist: ZZ Ward
Artist: Savoy Brown
Artist: Walter Trout
Artist: Black Stone Cherry
Found 1297 artists
Found 1297 artists
Artist: Our Lady Peace
Artist: The Tragically Hip
Artist: Aqua
Found 1300 artists
Found 1300 artists
Artist: Andrea Bocelli
Artist: Matthew Good Band
Artist: Lou Bega
Found 1303 artists
Found 1303 artists
Artist: Diana Krall
Found 1304 artists
Artist: Great Big Sea
Artist: Remy Shand
Artist: Shakira
Artist: Moby
Found 1308 artists
Artist: Deftone

...

Artist: Rae Sremmurd
Artist: O.T. Genasis
Found 5156 artists
Found 5156 artists
Artist: Kodak Black
Found 5157 artists
Found 5157 artists
Artist: Justin Timberlake Featuring JAY Z
Found 5158 artists
Artist: Jason Derulo Featuring Snoop Dogg
Found 5159 artists
Found 5159 artists
Found 5159 artists
Found 5159 artists
Artist: Sam Cooke
Artist: The Impressions
Artist: Jr. Walker & The All Stars
Artist: Ramsey Lewis Trio
Found 5163 artists
Artist: Jimmy Smith
Artist: Sam & Dave
Found 5165 artists
Artist: Cannonball Adderley Quintet
Found 5166 artists
Artist: O.C. Smith
Found 5167 artists
Artist: David Ruffin
Found 5168 artists
Artist: The Jackson 5
Found 5169 artists
Found 5169 artists
Found 5169 artists
Artist: The Spinners
Artist: The New Birth
Found 5171 artists
Artist: MFSB
Artist: James Brown
Artist: Spinners
Artist: Richard Pryor
Found 5175 artists
Artist: B.T. Express
Artist: Harold Melvin And The Blue Notes
Artist: Grover Washington, Jr.
Artist: Ramsey Lewis
Artist: Van Mccoy
Found 

Artist: Pebbles
Artist: Morris Day
Artist: Teena Marie
Artist: E.U.
Artist: The Mac Band Featuring The McCampbell Brothers
Artist: Rick James Featuring Roxanne Shante
Artist: Jeffrey Osborne
Artist: Cheryl Pepsii Riley
Artist: The Boys
Artist: Cherrelle
Artist: Ziggy Marley And The Melody Makers
Found 5448 artists
Artist: LeVert Feat. Heavy D
Artist: Today
Artist: Chuckii Booker
Artist: Teddy Riley Featuring Guy
Artist: Eric Gable
Artist: Miki Howard
Found 5454 artists
Artist: Ruby Turner
Artist: Stacy Lattisaw & Johnny Gill
Artist: Troop
Artist: After 7
Artist: Quincy Jones With Tevin Campbell
Artist: The Time
Artist: Samuelle
Found 5461 artists
Artist: The Rude Boys
Artist: Christopher Williams
Artist: Phil Perry
Artist: Lisa Fischer
Artist: Damian Dame
Artist: D.J. Jazzy Jeff & The Fresh Prince
Artist: Phyllis Hyman
Artist: Tracie Spencer
Artist: Shanice
Found 5470 artists
Artist: Gerald Levert (Duet With Eddie Levert)
Artist: Glenn Jones
Artist: R. Kelly & Public Announcement
Artis

Artist: Tanto Metro & Devonte
Found 5652 artists
Found 5652 artists
Found 5652 artists
Found 5652 artists
Artist: Elephant Man
Found 5653 artists
Artist: Kevin Lyttle
Artist: Skindred
Found 5655 artists
Artist: Hector "El Bambino"
Artist: Eliel
Artist: Luny Tunes & Baby Ranks
Artist: Damian "Jr. Gong" Marley
Found 5659 artists
Artist: Cham
Found 5660 artists
Artist: Stephen Marley
Artist: Collie Buddz
Artist: Ky-Mani Marley
Found 5663 artists
Artist: Morgan Heritage
Artist: Richie Spice
Artist: John Brown's Body
Artist: Beres Hammond
Found 5667 artists
Artist: Little Hero
Artist: Mishka
Artist: Mavado
Artist: Easy Star All-Stars
Artist: Ziggy Marley
Found 5672 artists
Found 5672 artists
Artist: Passafire
Artist: J Boog
Artist: The Green
Artist: Jimmy Cliff
Found 5676 artists
Artist: SOJA
Found 5677 artists
Artist: Fiji
Artist: Fortunate Youth
Artist: Christafari
Artist: Snoop Lion
Artist: Musiq Soulchild/Syleena Johnson
Artist: Gyptian
Artist: Makua Rothman
Found 5684 artists
Artist: P

Found 5840 artists
Found 5840 artists
Found 5840 artists
Artist: Trent Reznor And Atticus Ross
Found 5841 artists
Found 5841 artists
Found 5841 artists
Found 5841 artists
Found 5841 artists
Artist: Kurt Cobain
Found 5842 artists
Found 5842 artists
Found 5842 artists
Artist: Ylvis
Found 5843 artists
Found 5843 artists
Found 5843 artists
Found 5843 artists
Found 5843 artists
Found 5843 artists
Artist: Built To Spill
Artist: J.J. Cale & Eric Clapton
Found 5845 artists
Artist: Ben Harper And The Innocent Criminals
Found 5846 artists
Artist: Ryan Adams & The Cardinals
Found 5847 artists
Artist: Animal Collective
Artist: Yeah Yeah Yeahs
Artist: Sonic Youth
Found 5850 artists
Artist: John Legend & The Roots
Artist: Fistful Of Mercy
Artist: Belle And Sebastian
Found 5853 artists
Artist: Chris Cornell
Found 5854 artists
Artist: Sigur Ros
Artist: Neil Young & Crazy Horse
Found 5856 artists
Artist: Yo La Tengo
Artist: Jim James
Artist: Nick Cave & The Bad Seeds
Artist: Kurt Vile
Artist: Washed Ou

Artist: Julia Michaels
Artist: Syd
Artist: Gallant x Tablo x Eric Nam
Artist: Bibi Bourelly
Artist: Stargate Featuring P!nk & Sia
Artist: Cheat Codes Featuring Demi Lovato
Artist: SEVENTEEN
Artist: Lost Kings Featuring Ally Brooke & A$AP Ferg
Artist: Jax Jones Featuring Demi Lovato & Stefflon Don
Artist: GoldLink Featuring Brent Faiyaz & Shy Glizzy
Artist: Daniel Caesar Featuring Kali Uchis
Artist: RiceGum
Found 6119 artists
Artist: Demi Lovato Featuring Cher Lloyd
Artist: Naughty Boy Featuring Sam Romans
Artist: Jack & Jack
Found 6122 artists
Artist: Snoop Dogg Featuring Charlie Wilson
Artist: Fifth Harmony Featuring Kid Ink
Found 6124 artists
Artist: Little Mix Featuring Sean Paul
Artist: Zedd & Kesha
Artist: Little Mix Featuring Jason Derulo
Artist: Troye Sivan Featuring Alessia Cara
Found 6128 artists
Artist: Ariana Grande Featuring Future
Artist: Camila Cabello
Artist: Selena Gomez Featuring Gucci Mane
Artist: Louis Tomlinson Featuring Bebe Rexha & Digital Farm Animals
Artist: Cam

In [26]:
for artist,artistData in data.items():
    print(artist)
    for name,nameData in artistData.items():
        print("\t",name,'\t',len(categoryData))

Brook Benton
	 The Boll Weevil Song 	 1
	 It's Just A Matter Of Time 	 1
	 Thank You Pretty Baby 	 1
	 So Many Ways 	 1
	 Kiddio 	 1
	 Rainy Night In Georgia/Rubberneckin' 	 1
Connie Francis
	 Together 	 1
	 Don't Break The Heart That Loves You 	 1
	 Everybody's Somebody's Fool 	 1
	 My Heart Has A Mind Of Its Own 	 1
Joe Dowell
	 Wooden Heart 	 1
The Highwaymen
	 Michael 	 1
Bob Moore and His Orch.
	 Mexico 	 1
Sue Thompson
	 Sad Movies (Make Me Cry) 	 1
Jimmy Dean
	 Big Bad John 	 1
	 The First Thing Ev'ry Morning 	 1
	 The First Thing Ev'ry Morning (And The Last Thing Ev'ry Night) 	 1
Henry Mancini
	 Moon River 	 1
	 Love Theme From Romeo & Juliet 	 1
The Lettermen
	 When I Fall In Love 	 1
Elvis Presley
	 Can't Help Falling In Love 	 1
	 I'm Yours 	 1
	 The Wonder Of You 	 1
	 You Don't Have To Say You Love Me 	 1
	 My Boy 	 1
	 Aloha From Hawaii Via Satellite 	 1
	 Elv1s: 30 #1 Hits 	 1
	 Legendary Performer, Vol. 1 	 1
	 Promised Land 	 1
	 From Elvis Presley Blvd, Memphis, Tenne

	 Have You Never Been Mellow 	 1
	 Please Mr. Please 	 1
	 Something Better To Do 	 1
	 Let It Shine/He Ain't Heavy...He's My Brother 	 1
	 Come On Over 	 1
	 Don't Stop Believin' 	 1
	 Sam 	 1
	 Magic 	 1
	 If You Love Me Let Me Know 	 1
	 Let Me Be There 	 1
	 Physical 	 1
Chicago
	 Beginnings 	 1
	 Call On Me 	 1
	 Wishing You Were Here 	 1
	 If You Leave Me Now 	 1
	 Hard To Say I'm Sorry 	 1
	 You're The Inspiration 	 1
	 Look Away 	 1
	 Here In My Heart 	 1
	 Chicago V 	 1
	 Chicago VI 	 1
	 Chicago VII 	 1
	 Chicago VIII 	 1
	 Chicago IX: Chicago's Greatest Hits 	 1
	 The Very Best Of Chicago: Only The Beginning 	 1
Joan Baez
	 The Night They Drove Old Dixie Down 	 1
Cat Stevens
	 Peace Train 	 1
	 Morning Has Broken 	 1
	 Catch Bull At Four 	 1
Sonny & Cher
	 All I Ever Need Is You 	 1
	 I Got You Babe 	 1
David Cassidy
	 Cherish 	 1
Don McLean
	 American Pie (Parts I & II) 	 1
	 Wonderful Baby 	 1
	 American Pie 	 1
Nilsson
	 Without You 	 1
Roberta Flack
	 The First Time Ever

	 Show Her 	 1
	 Still Losing You 	 1
	 She Keeps The Home Fires Burning 	 1
	 Lost In The Fifties Tonight(In The Still) 	 1
	 Happy, Happy Birthday Baby 	 1
	 In Love 	 1
	 How Do I Turn You On 	 1
	 Snap Your Fingers 	 1
	 Where Do The Nights Go 	 1
	 Don't You Ever Get Tired (Of Hurting Me) 	 1
	 A Woman In Love 	 1
Dolly Parton
	 9 To 5 	 1
	 Pure & Simple 	 1
	 New Harvest...first Gathering 	 1
	 Here You Come Again 	 1
	 Heartbreaker 	 1
	 9 To 5 And Odd Jobs 	 1
	 Eagle When She Flies 	 1
	 Joshua 	 1
	 Jolene 	 1
	 I Will Always Love You 	 1
	 Love Is Like A Butterfly 	 1
	 The Bargain Store 	 1
	 It's All Wrong,but It's Alright/Two Doors Down 	 1
	 I Really Got The Feeling/baby I'm Burning 	 1
	 You're The Only One 	 1
	 Starting Over Again 	 1
	 Old Flames Can't Hold A Candle To You 	 1
	 But You Know I Love You 	 1
	 I Will Always Love You/do I Ever Cross Your Mind 	 1
	 Tennessee Homesick Blues 	 1
	 Think About Love 	 1
	 Why'd You Come In Here Lookin' Like That 	 1
	 Yell

	 Never Gonna Give You Up 	 1
	 It Would Take A Strong Strong Man 	 1
	 Cry For Help 	 1
	 Together Forever 	 1
Foreigner
	 I Don't Want To Live Without You 	 1
	 4 	 1
	 I Want To Know What Love Is 	 1
	 URGENT 	 1
	 Waiting For A Girl Like You 	 1
	 Say You Will 	 1
Johnny Hates Jazz
	 Shattered Dreams 	 1
George Michael
	 One More Try 	 1
	 Kissing A Fool 	 1
	 Faith 	 1
	 Monkey 	 1
	 Star People 	 1
	 Amazing 	 1
	 Flawless (Go To The City) 	 1
	 Father Figure 	 1
	 Praying For Time 	 1
Breathe
	 How Can I Fall? 	 1
Anita Baker
	 Giving You The Best That I Got 	 1
	 Rapture 	 1
	 Rhythm Of Love 	 1
	 My Everything 	 1
	 Just Because 	 1
Boy Meets Girl
	 Waiting For A Star To Fall 	 1
Sheriff
	 When I'm With You 	 1
Mike + The Mechanics
	 The Living Years 	 1
	 Silent Running 	 1
Roy Orbison
	 You Got It 	 1
	 Running Scared 	 1
The Bangles
	 Eternal Flame 	 1
	 Walk Like An Egyptian 	 1
Cher & Peter Cetera
	 After All (Love Theme From "Chances Are") 	 1
Thirty Eight Special
	 Seco

	 As I Lay Me Down 	 1
	 Tongues And Tails 	 1
Mariah Carey & Boyz II Men
	 One Sweet Day 	 1
Journey
	 When You Love A Woman 	 1
	 Escape 	 1
	 Journey's Greatest Hits 	 1
	 Separate Ways (Worlds Apart) 	 1
	 Revelation 	 1
Toni Braxton
	 Un-Break My Heart 	 1
	 Toni Braxton 	 1
	 The Heat 	 1
	 You're Makin' Me High/Let It Flow 	 1
	 I Don't Want To 	 1
	 Spanish Guitar 	 1
	 I Heart You 	 1
	 Seven Whole Days 	 1
	 Just Be A Man About It 	 1
	 Secrets 	 1
	 Pulse 	 1
	 He Wasn't Man Enough 	 1
	 Breathe Again 	 1
Kenny Loggins
	 For The First Time 	 1
	 Celebrate Me Home 	 1
	 Footloose 	 1
Jewel
	 You Were Meant For Me 	 1
	 You Were Meant For Me/Foolish Games 	 1
	 Picking Up The Pieces 	 1
	 Perfectly Clear 	 1
	 Serve The Ego 	 1
	 Intuition 	 1
	 Stand 	 1
	 Lullaby 	 1
Bob Carlisle
	 Butterfly Kisses 	 1
	 Butterfly Kisses (Shades Of Grace) 	 1
Shawn Colvin
	 Sunny Came Home 	 1
LeAnn Rimes
	 How Do I Live 	 1
	 Unchained Melody/The Early Years 	 1
	 You Light Up My Life -- In

...

	 Quiereme Mas 	 1
	 Divinas 	 1
	 Tierra Extrana 	 1
	 Sin Ti No Vive 	 1
	 Como Pude Enamorarme De Ti 	 1
	 Eres Divina 	 1
La Quinta Estacion
	 Sin Frenos 	 1
	 Que Te Queria 	 1
Ednita
	 Soy 	 1
The Chieftains Featuring Ry Cooder
	 San Patricio 	 1
El Trono de Mexico
	 Quiero Decirte Que Te Amo 	 1
	 Almas Gemelas 	 1
	 Lo Mejor de El Trono de Mexico 	 1
	 A Corazon Abierto 	 1
Gloria Trevi
	 Gloria 	 1
	 El Amor 	 1
	 Inmortal 	 1
Jencarlos Canela
	 Un Nuevo Dia 	 1
	 Buscame 	 1
Christian Pagan
	 De Miles A Uno 	 1
Roberto Tapia
	 El Muchacho 	 1
	 Lo Mejor de Roberto Tapia 	 1
	 Mi Nina 	 1
	 Diferente 	 1
	 Mirando Al Cielo 	 1
Tommy Torres
	 12 Historias 	 1
Tierra Cali
	 Entregate 	 1
	 Un Siglo De Amor 	 1
	 Romanticos del Momento 	 1
Julion Alvarez y Su Norteno Banda
	 Tu Amigo Nada Mas 	 1
	 El Aferrado 	 1
	 Mis Idolos, Hoy Mis Amigos!!! 	 1
	 En Vivo 	 1
	 Ni Diablo Ni Santo 	 1
	 Ni Lo Intentes 	 1
	 Olvidame 	 1
	 Te Hubieras Ido Antes 	 1
	 Y Asi Fue 	 1
	 El Amor de 

	 Make It Funky (Part 1) 	 1
	 Talking Loud And Saying Nothing - Part I 	 1
	 Get On The Good Foot-Part 1 	 1
	 THE PAYBACK (PART I) 	 1
	 My Thang 	 1
	 Papa Don't Take No Mess (Part I) 	 1
Spinners
	 Mighty Love 	 1
	 New And Improved 	 1
	 Could It Be I'm Falling In Love 	 1
Richard Pryor
	 That Nigger's Crazy 	 1
	 Is It Something I Said? 	 1
B.T. Express
	 Do It 'til You're Satisfied 	 1
	 Non-stop 	 1
	 Do It ('Til You're Satisfied) 	 1
	 Express 	 1
Harold Melvin And The Blue Notes
	 To Be True 	 1
	 Wake Up Everybody 	 1
	 If You Don't Know Me By Now 	 1
	 The Love I Lost (Part 1) 	 1
	 Wake Up Everybody (Part 1) 	 1
Grover Washington, Jr.
	 Mister Magic 	 1
	 Feels So Good 	 1
Ramsey Lewis
	 Sun Goddess 	 1
Van Mccoy
	 Disco Baby 	 1
Rufus And Chaka Khan
	 Rufus Featuring Chaka Khan 	 1
	 Ask Rufus 	 1
	 Street Player 	 1
	 Masterjam 	 1
	 Do You Love What You Feel 	 1
Brass Construction
	 Brass Construction 	 1
	 Movin' 	 1
Brick
	 Good High 	 1
	 Brick 	 1
	 DAZZ 	 1
Bootsy'

	 Diamonds 	 1
Alexander O'Neal
	 Fake 	 1
Force M.D.'s
	 Love Is A House 	 1
Angela Winbush
	 Angel 	 1
Pebbles
	 Girlfriend 	 1
	 Mercedes Boy 	 1
	 Giving You The Benefit 	 1
	 Love Makes Things Happen 	 1
Morris Day
	 Fishnet 	 1
Teena Marie
	 Ooo La La La 	 1
E.U.
	 Da'Butt (From The "School Daze" Soundtrack) 	 1
The Mac Band Featuring The McCampbell Brothers
	 Roses Are Red 	 1
Rick James Featuring Roxanne Shante
	 Loosey's Rap 	 1
Jeffrey Osborne
	 She's On The Left 	 1
Cheryl Pepsii Riley
	 Thanks For My Child 	 1
The Boys
	 Dial My Heart 	 1
	 Lucky Charm 	 1
	 Crazy 	 1
Cherrelle
	 Everything I Miss At Home 	 1
Ziggy Marley And The Melody Makers
	 Tumblin' Down 	 1
	 Spirit Of Music 	 1
LeVert Feat. Heavy D
	 Just Coolin' 	 1
Today
	 Girl I Got My Eyes On You 	 1
Chuckii Booker
	 Turned Away 	 1
	 Games 	 1
Teddy Riley Featuring Guy
	 My Fantasy (From "Do The Right Thing") 	 1
Eric Gable
	 Remember (The First Time) 	 1
Miki Howard
	 Ain't Nuthin' In The World 	 1
	 Ain't Nobo

	 Antes Muertas Que Sencillas 	 1
	 Viejitas Pero Buenas... Pa' Pistear 	 1
	 Dos Locos 	 1
Beto y Sus Canarios
	 Ardientes 	 1
	 Esta Llorando Mi Corazon 	 1
	 No Puedo Olvidarte 	 1
Graciela Beltran
	 Rancherisimas Con Banda 	 1
	 Es Cosa De El 	 1
Ramon Ayala y Sus Bravos del Norte
	 Antologia De Un Rey 	 1
Jose Alfredo Jimenez
	 La Historia Del Rey 	 1
Diana Reyes
	 Las No. 1 De La Reina 	 1
	 Insatisfecha 	 1
Brazeros Musical de Durango
	 Rosas Rojas 	 1
Javier Solis
	 La Historia De Javier Solis 	 1
Pesado
	 Piensame Un Momento 	 1
	 Mi Promesa 	 1
	 Directo 93-13, Vol. 1 	 1
	 Abrazame 	 1
	 Ojala Que Te Mueras 	 1
	 El Mil Amores 	 1
El Chapo de Sinaloa
	 Te Va A Gustar 	 1
	 La Noche Perfecta 	 1
	 A Ti Si Puedo Decirte 	 1
Martinez Cruz Presenta Los Super Reyes
	 El Regrso De Los Reyes 	 1
Juan Gabriel & Ana Gabriel
	 Los Gabriel...Cantan A Mexico 	 1
German Montero
	 Comprendeme 	 1
	 Amantes Escondidos 	 1
Conjunto Atardecer
	 Contigo Para Siempre 	 1
	 Solo Junto A Ti 	 1


Aymee Nuviola Featuring Baby Rasta & Gringo
	 Bailando Todo Se Olvida 	 1
Xantos
	 Bailame Despacio 	 1
Alejandra
	 El Rey Estafador 	 1
Gabo Parisi Featuring Farruko
	 Muneca 	 1
Domino Saints
	 Ya Quiero 	 1
Jory Boy
	 Detras de Ti 	 1
Diego Amador Featuring Oscar D'Leon
	 La Sandunguita 	 1
Prince Royce Featuring Gerardo Ortiz
	 Moneda 	 1
Alex Sensation + Gente de Zona
	 La Mala y La Buena 	 1
Prince Royce & Shakira
	 Deja Vu 	 1
Nacho
	 Bailame 	 1
Romeo Santos Featuring Nicky Jam & Daddy Yankee
	 Bella y Sensual 	 1
KCamp Featuring 2 Chainz
	 Cut Her Off 	 1
Buddy Featuring Miley Cyrus
	 Smoke Signals 	 1
Gorgon City Featuring Laura Welsh
	 Here For You 	 1
Pierce Fulton
	 Runaway 	 1
Fences Featuring Macklemore
	 Arrows 	 1
P Reign Featuring Drake & Future
	 DnF 	 1
INFINITE
	 Last Romeo 	 1
P Reign Featuring Meek Mill & PARTYNEXTDOOR
	 Realest In The City 	 1
OG Maco Featuring 2 Chainz
	 U Guessed It 	 1
DeJ Loaf
	 Try Me 	 1
MisterWives
	 Reflections 	 1
Gorgon City Featuring 

In [18]:
dtval = 'December 25, 1962'
from timeUtils import getDateTime, isDate, isDateTime
x = getDateTime(dtval)
print(x)
print(isDate(x))
print(isDateTime(x))

1962-12-25 00:00:00
True
True


In [ ]:



#years = ['1990']
for year in years:
    for name,data in chartData.items():
        base = getDirname(data[1])
        url = join(data[0], join(base, year), data[2])
        savedir = join(basedir, "data", "billboard")
        savename = ".".join(["-".join([year, data[2]]), 'p'])
        savename = join(savedir, savename)
        
        #print(isFile(savename), savename)
        if isFile(savename):
            continue
        
        print(url)

        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,} 

        try:
            request=urllib.request.Request(url,None,headers) #The assembled request
            response = urllib.request.urlopen(request)
            data = response.read() # The data u need
        except:
            continue
        
        print("Saving {0}".format(savename))
        saveJoblib(data=data, filename=savename, compress=True)
        sleep(1)
        continue
        
        getWebData(base=url, savename=savename, useSafari=False)
        try:
            getWebData(base=url, savename=savename, useSafari=False)
        except:
            pass

In [71]:
import urllib
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

url = "http://en.wikipedia.org/wiki/List_of_TCP_and_UDP_port_numbers"
headers={'User-Agent':user_agent,} 

request=urllib.request.Request(url,None,headers) #The assembled request
response = urllib.request.urlopen(request)
data = response.read() # The data u need

In [82]:
data = loadJoblib("/Users/tgadfort/Documents/code/charts/data/billboard/1958-alternative-songs.p")
data

b'<!doctype html>\n<html class="" lang="">\n<head>\n<meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=device-width, initial-scale=1, user-scalable=no">\n<title>1958 Archive | Billboard</title>\n<meta name="description" property="description" content="" />\n<meta name="title" property="title" content="1958 Archive" />\n<meta name="twitter:site" content="@billboard">\n<meta property="og:site_name" content="Billboard" />\n<meta property="og:type" content="article" />\n<link rel="canonical" href="https://www.billboard.com/archive/charts/1958/alternative-songs">\n<link rel="shortcut icon" href="https://assets.billboard.com/assets/1548962659/images/favicon.ico?52b3f835b807020ba548" type="image/vnd.microsoft.icon" />\n<link rel="apple-touch-icon" href="https://assets.billboard.com/assets/1548962659/images/BB_favicon144.png?52b3f835b807020ba548">\n<script type="text/javascript">\n        var PGM = window.PGM || {};\n    PGM.conf

In [65]:
#col.createCollectionsData()
#col.downloadCollections()
#col.parseCollections()
#col.mergeCollections(debug=True)
col.createCollectionDBs(debug=True)

Loading data from /Volumes/Music/Discog/collections-db/mergedParsedCollections.p
  --> This file is 45.4MB.
Loading /Volumes/Music/Discog/collections-db/mergedParsedCollections.p
Found 1083782 artists from /Volumes/Music/Discog/collections-db/mergedParsedCollections.p
Current Time is Mon Jan 21, 2019 19:01:15 for Creating Initial Artist Database
Not all values are real for /artist/Unknown+Artist?anv=Unknown+Artists
Not all values are real for /artist/66827-
Not all values are real for /artist/Unknown+Artist?anv=%D9%85%D8%B3%D8%B9%D9%88%D8%AF+%D8%B3%D8%A7%D9%84%D9%85+%D9%85%D9%8A%D8%A7%D8%B3%D8%A9
Not all values are real for /artist/Various?anv=Musicians+Against+Childhood+Cancer
Not all values are real for /artist/Unknown+Artist?anv=Country+Music%27s+Hottest+Pickers
Not all values are real for /artist/Anke%2BAngel
Not all values are real for /artist/Various?anv=Razni+Izvo%C4%91a%C4%8Di
Not all values are real for /artist/Unknown+Artist?anv=Ein+Soldatenchor
Not all values are real for /a

# Artist Section

In [77]:
art = artists(disc)
art.getArtistsData()

0 /artist/82730-The-Beatles
1 /artist/10263-David-Bowie
2 /artist/23755-Miles-Davis
3 /artist/92623-Frank-Zappa
4 /artist/27518-Elvis-Presley
5 /artist/8760-Madonna
6 /artist/20991-The-Rolling-Stones
7 /artist/15885-Michael-Jackson
8 /artist/59792-Bob-Dylan
9 /artist/45467-Pink-Floyd
10 /artist/28795-Prince
11 /artist/66852-The-Prodigy
12 /artist/95537-Johann-Sebastian-Bach
13 /artist/15900-Ennio-Morricone
14 /artist/18839-Metallica
15 /artist/97545-John-Coltrane
16 /artist/2725-Depeche-Mode
17 /artist/81013-Queen
18 /artist/135946-Johnny-Cash
19 /artist/52833-Frank-Sinatra
20 /artist/28972-The-Cure
21 /artist/11769-Scooter
22 /artist/30724-Santana
23 /artist/21764-Bob-Marley-amp-The-Wailers
24 /artist/57103-Elton-John
25 /artist/70829-The-Beach-Boys
26 /artist/34278-Led-Zeppelin
27 /artist/10783-Beastie-Boys
28 /artist/2290-The-Chemical-Brothers
29 /artist/6520-U2
30 /artist/125246-Nirvana
31 /artist/175395-Pearl-Jam
32 /artist/3865-Herbie-Hancock
33 /artist/273394-John-Williams-4
34 

ZeroDivisionError: division by zero

In [ ]:




################################################################################
#
# Download Artist Files
#
################################################################################
def downloadArtists(minCounts = 1, debug = False, forceWrite = False):
    basedir      = getBaseDBDir()    
    countsname   = setFile(basedir, "artistFrequency.yaml")
    artistCounts = Counter(get(countsname))
    
    discdbname   = setFile(basedir, "downloadedArtists.yaml")
    discdb       = get(discdbname)
    

    print "Finding artists to download..."
    artistsToGet = []
    for item in artistCounts.most_common():
        href = makeUnicode(item[0])
        cnt  = item[1]
        if cnt < minCounts: continue
        discID = getArtistDiscID(href)
        if discdb.get(discID): continue
        artistsToGet.append(href)

    print "Finding artists to download...",len(artistsToGet)
        
    if debug: print "Loading",dbname
    dbdata = get(dbname)
    if debug: print "Found",len(dbdata),"unique artists from",dbname

    dbname  = setFile(basedir, "artistDB.json")
    if debug: print "Loading",dbname
    db      = get(dbname)
    if debug: print "Found",len(db),"known artists from",dbname
    
    errDB = getBadIDs(getDiscogDir())

    downloads = {}
    downloads = getMissing(dbdata, db, errDB)    
    if debug: print "Downloading",len(downloads),"new artists for missing artists."


    if len(downloads) == 0:
        print "Looping over",len(dbdata),"artist counts data..."
        for k,v in dbdata.iteritems():
            if v["CNT"] < minCounts:
                continue
            discID = getArtistDiscID(v["URL"])
            if db.get(discID):
                continue
            if errDB.get(discID):
                continue
            downloads[k] = v
            
        if debug:
            print "Downloading",len(downloads),"new artists with >=",minCounts,"counts."


    
    baseurl = u"https://www.discogs.com"
    useSafari = True
    dtime = 4
    problems = {}
    
    curDir  = setDir(base, artistDir)
    curDirs = [x for x in findAll(curDir) if isDir(x)]
    subDir  = setDir(base, artistDir)
    savedir = mkSubDir(subDir, str(len(curDirs)))
    print "\n\n===========> Saving files to",savedir," <===========\n\n"
    
    for i,artist in enumerate(downloads.keys()):
        v = downloads[artist]
        if i % 10 == 0:
            print '\n',i,'/',len(downloads),'\t',artist,'\t',v,'\n'
        suburl = v["URL"]
        artistSaveName = getSaveName(artist)
        savename = join(savedir, artistSaveName+"-1.p")
        if isFile(savename):
            continue
        retval = getData(baseurl, suburl, extra=None, savename=savename, 
                         useSafari=useSafari, dtime=dtime, debug=debug)
        if not retval:
            problems[artist] = suburl


    print "Found",len(problems),"download problems."
    if len(problems) > 0:
        savename = setFile(getDiscogDir(), "problems.json")
        print "Saving them to",savename
        save(savename, problems)
    



################################################################################
#
# Process the ArtistDB Files
#
################################################################################
def processArtistDBFiles(debug = False):
    basedir = getBaseDBDir()
    
    artistCntr   = Counter()    
    artistCntrDB = {}

    if debug:
        print "Looking for files in",basedir
    files = findPatternExt(basedir, pattern="artistDB-", ext=".p")
    #files = glob(join(basedir, "artistDB-*.p"))
    if debug:
        print "Found",len(files),"files in",basedir
                         
    nerr = 0
    print "  ",nice("   Progress", 20),nice("#Artists", 10),nice("#Errors", 10),"Filename"
    for i,ifile in enumerate(files):
        if i % 1000 == 0 or i == 10 or i == 100:
            print "  ",nicerate(i,len(files), 20),
            print nice(len(artistCntr), 10),
            print nice(nerr, 10),
            print ifile
        data = get(ifile)
        
        for artist,suburl in data.iteritems():
            discID = getArtistDiscID(suburl)
            if discID == None:
                if debug:
                    nerr += 1
                    #print "    No discID for",artist,suburl
                continue
            artistCntr[artist] += 1
            if artistCntrDB.get(artist) == None:
                artistCntrDB[artist] = suburl

    if debug:
        print "Found ",len(artistCntrDB),"unique artists from files in",basedir

    savename = setFile(basedir, "artistCountsAll.p")
    print "Saving",len(artistCntrDB),"artists to",savename
    save(savename, artistCntrDB)
    print savename,'size ->',getSize(savename, unit='MB'),"MB."

    savename = setFile(basedir, "artistCountsRaw.p")
    print "Saving",len(artistCntr),"artists to",savename
    save(savename, artistCntr)
    print savename,'size ->',getSize(savename, unit='MB'),"MB."


    
def findArtists(minCounts = 1, debug = True):
    basedir = getDiscogBaseDBDir()

    savename = setFile(basedir, "artistCountsAll.p")
    if debug: print "Loading",savename
    artistCntrDB = get(savename)
    if debug: print "Found",len(artistCntrDB),"artists in",savename
    
    savename = setFile(basedir, "artistCountsRaw.p")
    if debug: print "Loading",savename
    artistCntr = get(savename)
    if debug: print "Found",len(artistCntr),"artists in",savename
    

    artistCounter = {}                 
    for artist in artistCntrDB.keys():
        val = artistCntr[artist]
        if val < minCounts:
            continue
        url = artistCntrDB[artist]
        artistCounter[artist] = {"URL": url, "CNT": val}
        
        
    savename = setFile(basedir, "artistCounts.p")
    print "Saving",len(artistCounter),"/",len(artistCntrDB),
    print "unique artists with >=",minCounts,"counts to",savename
    save(savename, artistCounter)
    print savename,'size ->',getSize(savename, unit='kB'),"kB."
    
                                    



################################################################################
#
# Artist Helpers
#
################################################################################
def saveArtistData(artistData, debug = False, ifile = None, forceWrite = False):
    artistDBDir = getArtistsDBDir()

    artist   = artistData["Artist"]
    discID   = artistData["ID"]
    if artist == None or discID == None:
        removeFile(ifile)
        print " --> Removing due to artist/discID error:",ifile
        return

    modValue = getDiscIDHashMod(discID, modval=500)
    subDir   = mkSubDir(artistDBDir, str(modValue))
    outdir   = subDir
    savename  = setFile(outdir, discID+".p")
    if isFile(savename) and not forceWrite:
        return
    save(savename, artistData)
    print " --> Saved",savename




################################################################################
#
# Special Artists
#
################################################################################
def parseSpecialArtists(base = "/Volumes/Music/Discog", debug = False):  
    basedbdir   = getDiscogBaseDBDir()        
    dbname  = setFile(basedbdir, "artistDB.json")
    dbdata  = get(dbname)
    
    specialdir  = getDiscogSpecialDir()
    files = findExt(specialdir, ext=".html")
    for i,ifile in enumerate(files):
        if i % 250 == 0 or i == 50 or i == 10:
            print "\n====>",i,"/",len(files),'\t',ifile
        if getsize(ifile) < 1000:
            print " --> Removing due to low size:",ifile
            removeFile(ifile)
            continue
        
        bsdata         = getHTML(get(ifile))
        artistData     = parse(bsdata, debug)
        saveArtistData(artistData, dbdata, debug, ifile, forceWrite = False)

        artist         = artistData["Artist"]
        artistSaveName = getSaveName(artist)
        savename = setSubFile(base, "artists-special", artistSaveName+"-1.p")
        save(savename, open(ifile).read())

        if isFile(ifile):
            if debug:
                print " --> Removing special artist:",ifile
            removeFile(ifile)




################################################################################
#
# Special Artists
#
################################################################################
def downloadMultipageArtist(debug = False, forceWrite = False):
    
def parseMultipageArtists(debug = False, forceWrite = False):
    files = findPatternExt(getArtistsExtraDir(), pattern='-1', ext='.p')
    artists = [x.replace("-1.p", "") for x in files]
    discIDs = [getBasename(x) for x in artists if x.endswith('.p') == False]
    
    for i,discID in enumerate(discIDs):
        print i,'/',len(discIDs)
        files = findPatternExt(getArtistsExtraDir(), pattern=discID+"-", ext='.p')
        fullArtistData = None
        print "  Found",len(files),"for discID:",discID
        for j,ifile in enumerate(files):
            print "    -->",j,"/",len(files)
            bsdata         = getHTML(ifile)
            artistData     = artistdata.parse(bsdata, debug)
        
            if j == 0:
                fullArtistData = artistData
                continue
            else:
                #fullArtistData["Pages"] = max(int(fullArtistData["Pages"]), int(artistData["Pages"]))
                for media,mediaData in artistData["Media"].iteritems():
                    if fullArtistData["Media"].get(media) == None:
                        fullArtistData["Media"][media] = mediaData
                    else:
                        for item in mediaData:
                            fullArtistData["Media"][media].append(item)

        
        modValue = getDiscIDHashMod(discID, modval=500)
        dbname = setFile(getArtistsDBDir(), str(modValue)+"-DB.p")
        dbdata = get(dbname, debug)
        dbdata[discID] = fullArtistData
        save(dbname, dbdata, debug = True)
        
    


################################################################################
#
# Artist Data
#
################################################################################
def parseArtistFile(ifile, debug = False, forceWrite = False):
    retval = checkFileSize(ifile, 1000)
    if not retval: return None
    
    bsdata     = getHTML(get(ifile))
    artistData = artistdata.parse(bsdata) 
    return artistData
    
    
def parseArtists(dirnum, artistdir = "artists",
                 debug = False, forceWrite = False):
    base = getDiscogDir()
    basedbdir = getBaseDBDir()
    artistdir = setDir(base, artistdir)
    
    
    basedir = setDir(artistdir, str(dirnum))
    if not isDir(basedir):
        if debug: print basedir,"does not exist."
        return

    dbname  = setFile(basedbdir, "artistDB.json")
    dbdata  = get(dbname)

    files = findExt(basedir, ext=".p")
    print "Found",len(files),"in",basedir
    for i,ifile in enumerate(files):
        if i % 25 == 0 or i == 50 or i == 10:
            print "\n====>",i,"/",len(files),'\t',ifile
        artistData = parseArtistFile(ifile, debug, forceWrite)
        if artistData:
            saveArtistData(artistData, dbdata, debug, ifile, forceWrite)




################################################################################
#
# Show ArtistData
#
################################################################################
def showArtistData(artistData):
    print nice("Artist:",10),artistData.get("Artist")
    for key in artistData.keys():
        print key




################################################################################
#
# Update Artist DBs
#
################################################################################
def updateArtistDBs():
    artistDBDir = getArtistsDBDir()
    files = findExt(artistDBDir, ext=".p")
    for ifile in files:
        data = get(ifile)
        for discID,artistData in data.iteritems():
            media = artistData['Media']
            for mediatype in media.keys():
                tmp = {}
                for item in media[mediatype]:
                    code = item['Code']
                    del item['Code']
                    tmp[code] = item
                media[mediatype] = tmp
                     
        save(ifile, data)



################################################################################
#
# Error handlers
#
################################################################################
def getBadIDs(base, debug = True):
    errDBname = setSubFile(base, "artists-db-err", "errDB.json")
    errDB = get(errDBname)
    
    if debug: print "  Found ",len(errDB),"bad IDs."
    
    files = findSubExt(base, "artists-db-err", ext=".p")
    #for ifile in glob(join(base, "artists-db-err", "*.p")):
    for ifile in files:
        data = get(ifile)
        try:
            discID = data["ID"]
            errDB[discID] = 1
        except:
            continue
        
        removeFile(ifile)

    if debug: print "  Saving",len(errDB),"bad IDs."
    save(errDBname, errDB)
    return errDB
    

    
def removeKnownArtists(allDB, known, debug = False):
    print "Removing Known Artists:",len(allDB)
    print "         Known Artists:",len(known)
    for artist in known.keys():
        if allDB.get(artist):
            del allDB[artist]
    print "     New Known Artists:",len(allDB)
    raise ValueError("Done.")
    return allDB




def getMissing(dbdata, db, errDB, debug = False):
    missingDir  = mkSubDir(getMusicDir(), "missing")
    missingFile = setFile(missingDir, "missing.json")
    missing     = get(missingFile)
    
    downloads = {}
    #allDB = removeKnownArtists(dbdata, db)
    artists = dbdata.keys()
    #artists = allDB.keys()
    
    for i,artist in enumerate(reversed(missing.keys())):
        if len(artist) < 2: continue
        name = artist.replace("The ", "")        
        matches1 = findMatchingWord(name, artists)
        #matches2 = []
        matches2 = findNearest(name, artists, 100, 0.75)
        matches  = list(set(matches1 + matches2))
        #matches  = matches[:min(len(matches), 10)]
        if len(matches) > 0:
            print i,'/',len(missing),'\t',len(downloads),'\t',artist,' \t---> ',len(matches)
        for x in matches:
            v = dbdata[x]
            discID = getArtistDiscID(v["URL"])
            if db.get(discID):
                continue
            if errDB.get(discID):
                continue
            downloads[x] = v
        
        if len(downloads) > 100000:
            break
        
    return downloads



def splitArtistFilesByHashval(artistdir = "artists", N = 2000):
    base   = getDiscogDir()
    files  = glob(join(base, "artistFiles", "artists[0-9]", "*.p"))
    files += glob(join(base, "artistFiles", "artists[1-9][0-9]", "*.p"))
    print "Found",len(files)

    modN  = int(ceil(len(files) / N)) + 2
    fvals = {}    
    for ifile in files:
        hexval = int(sha1(ifile).hexdigest(), 16)
        modval = hexval % modN
        if fvals.get(modval) == None:
            fvals[modval] = []
        fvals[modval].append(ifile)
        
    for k,v in fvals.iteritems():
        outdir = mkSubDir(base, [artistdir, str(k)])
        print "Moving",len(v),"to",outdir
        for ifile in v:
            src = ifile
            dst = setFile(outdir, getBasename(src))
            moveFile(src, dst)
            
            
            


def addNewArtistsDBToDB():
    startVal       = start()
    artistDB       = getArtistDB()
    tmpdir = setDir(getDiscogDir(), "artists-db")
    files = findSubExt(getDiscogDir(), "artist-db", ext=".p")
    newToDB  = {}
    for i,ifile in enumerate(files):
        if (i+1) % 25 == 0: inter(startVal, i+1,len(files))
        artistData = get(ifile)
        #artistData = parseArtistFile(ifile)
        #print artistData
        discID = artistData.get("ID")
        if discID == None:
            removeFile(ifile, debug = True)
            continue
        if newToDB.get(discID):
            removeFile(ifile, debug = True)
            continue
        if artistDB.get(discID) == None:
            ref    = artistData.get("URL")
            name   = makeStrFromUnicode(makeUnicode(artistData.get("Artist")))
            newToDB[discID] = {"URL": ref, "Name": name}

        outfile = setFile(tmpdir, discID+".p")
        if isFile(outfile):
            removeFile(ifile, debug = True)
            continue
        moveFile(ifile, outfile, debug = True)
        
            
    saveNewDBs(newToDB)
    end(startVal)



def addNewArtistsToDB():
        
    startVal       = start()
    artistDB       = getArtistDB()
    knownArtistIDs = getKnownArtistIDs()
    
    tmpdir = setDir(getDiscogDir(), "artists-tmp")
    files = findSubExt(getDiscogDir(), "artists-special", ext=".p")
    newToDB  = {}
    for i,ifile in enumerate(files):
        if (i+1) % 25 == 0: inter(startVal, i+1,len(files))
        artistData = parseArtistFile(ifile)
        #print artistData
        discID = artistData.get("ID")
        if discID == None:
            removeFile(ifile, debug = True)
            continue
            
        if knownArtistIDs.get(discID) or newToDB.get(discID):
            removeFile(ifile, debug = True)
            continue
        
        outfile = setFile(tmpdir, discID+".p")
        moveFile(ifile, outfile, debug = True)

        if artistDB.get(discID) == None or True:
            ref    = artistData.get("URL")
            name   = makeStrFromUnicode(makeUnicode(artistData.get("Artist")))
            newToDB[discID] = {"URL": ref, "Name": name}
        #print ifile,'\t\t',discID,'\t',ref,'\t\t',name

    saveNewDBs(newToDB)
    end(startVal)



###############################################################################
#
# Re-arrange Artists by ModVal
#
###############################################################################
def moveArtistsByHash():
    artistsDir = getArtistsDBDir()
    #artistNameDB = getArtistNameDB(slim = False, debug = True)
    #files = findSubExt(setDir(getDiscogDir(), "artistNew"), "*", ext=".p")
    files = findSubExt(getDiscogDir(), "artists-db", ext=".p")
    #cnts = [0, 0]
    for ifile in files:
        discID = getBaseFilename(ifile)
        modValue = getDiscIDHashMod(discID, modval=500)
        subDir   = mkSubDir(artistsDir, str(modValue))
        outdir   = subDir
        outname  = setFile(outdir, discID+".p")
        if isFile(outname):
            removeFile(ifile, debug = True)
            continue
        moveFile(ifile, outname, debug = True)
        

In [ ]:




###############################################################################
#
# Search Results
#
###############################################################################
def downloadSearchResults(searchTerm, forceWrite = False, debug = True):
    if debug:
        print "downloadSearchResults(",searchTerm,")"
    baseURL   = u"https://www.discogs.com/search/"
    #subURL    = "?limit=250&q="+pathname2url(makeUnicode(searchTerm).encode("utf-8"))+"&type=artist&layout=sm"
    subURL    = "?limit=250&q="+pathname2url(makeUnicode(searchTerm).encode("utf-8"))+"&layout=sm"
    #subURL    = "?q="+searchTerm+"&type=artist&layout=sm"
    URL       = baseURL + subURL
    savename  = setFile(getSearchDir(),getSaveName(searchTerm)+".p")
    if isFile(savename) and forceWrite == False:
        return
    
    if debug:
        print "Saving search results for",searchTerm,"to",savename
    dtime     = 5
    useSafari = True    
    attempts  = 1
    retval    = False
    
    while not retval and attempts <= 3:
        retval = getData(base=URL, suburl=None, extra=None, savename=savename, 
                         useSafari=useSafari, dtime=dtime, debug=debug)
        if not retval:
            print "  There was an error. Logging it."
            attempts += 1
            if isFile(savename):
                removeFile(savename, debug)

    if retval and debug:
        print "Downloaded search results for",searchTerm


        
def parseSearchResultsFile(bsdata):
    refDB    = {}
    for h4 in bsdata.findAll("h4"):
        ref = h4.find("a")
        if ref:
            attrs  = ref.attrs
            href   = attrs.get('href')
            if href:
                if href.find("anv=") != -1:
                    continue
            discID = getArtistDiscID(href)
            
            refDB[discID] = href
                     
    return refDB




def parseDownloadedSearchResults(forceWrite = False, debug = True):
    files = findExt("/Volumes/Music/Discog/search", ".html")
    for ifile in files:
        savename = ifile.replace(".html", ".p")
        data = open(ifile).read()
        save(savename, data)    
    
    #artistDB = getArtistDB()
    files    = findExt(getSearchDir(), ext=".p")

    toGet = {}
    print "Searching through",len(files),"search files."
    startVal = start()
    for i,ifile in enumerate(files):
        if (i+1) % 25 == 0: inter(startVal, i+1, len(files))
        bsdata = getHTML(ifile)
        refDB = parseSearchResultsFile(bsdata)
        for discID,href in refDB.iteritems():
            if toGet.get(discID): continue
            #if artistDB.get(discID) or toGet.get(discID): continue
            toGet[discID] = href

    end(startVal)

    savename = setFile(getSearchDir(), "toGet.yaml")    
    print "Downloading",len(toGet),"new disc IDs."
    save(savename, toGet, debug = True)
    


def downloadSearchResultArtists(debug = False, dtime = 4):
    artistDB = getArtistDB()
    outdir   = getSearchArtistsDir()
    outdbdir = getSearchArtistsDBDir()
    baseURL  = u"https://www.discogs.com"
    newToDB  = {}
    useSafari = True
    
    
    savename = setFile(getSearchDir(), "toGet.yaml")
    toGet = get(savename, debug = True)
    print "Downloading",len(toGet),"new disc IDs."

    startVal = start()

    for i,discID in enumerate(toGet.keys()):
        
        if (i+1) % 100 == 0: inter(startVal, i+1, len(toGet))
        
        print i,'/',len(toGet)
        if discID == None:
            continue
    
        href = toGet[discID]
        
        if artistDB.get(discID) or newToDB.get(discID): continue
        #if newToDB.get(discID): continue

        try:
            savename = setFile(outdir, discID+".p")
        except:
            continue
        
        if isFile(savename):
            continue
        
        URL = baseURL + href
        URL = baseURL + pathname2url(makeUnicode(href).encode("utf-8"))
        URL = URL + "?sort=year%2Casc&limit=500&page=1"

        retval   = False
        attempts = 0
        while not retval and attempts < 3:
            retval = getData(base=URL, suburl=None, extra=None, savename=savename, 
                             useSafari=useSafari, dtime=dtime+2*attempts, debug=debug)
            attempts += 1
            #sleep()
            
        if retval:
            bsdata = getHTML(savename)
            artistData     = parse(bsdata, debug)
            savename = setFile(outdbdir, discID+".p")
            if isFile(savename):
                continue
            save(savename, artistData, debug = True)

        newToDB[discID] = 1
               
    end(startVal)
               

def mergeSearchResults(debug = True):
    newToDB  = {}
    artistDBData = {}

    modVal   = 500
        
    files    = findExt(getSearchArtistsDBDir(), ext=".p")
    for i,ifile in enumerate(files):
        if i % 100 == 0:
            print i,'/',len(files),'\t',ifile
        artistData = get(ifile)
        discID   = artistData["ID"]
        href     = artistData["URL"]
        artist   = makeStrFromUnicode(artistData["Artist"])
        modValue = getDiscIDHashMod(discID, modval=modVal)
        if artistDBData.get(modValue) == None:
            artistDBData[modValue] = {}
        newToDB[discID] = {"URL": href, "Name": artist}
        artistDBData[modValue][discID] = artistData

    saveNewDBs(newToDB)    
    mergeArtistDBs(False)

    for modValue in artistDBData.keys():
        modDBfile = setFile(getArtistsDBDir(), str(modValue)+"-DB.p")
        modDB = get(modDBfile)
        for discID in artistDBData[modValue].keys():
            modDB[discID] = artistDBData[modValue][discID]
        try:
            save(modDBfile, modDB)
        except:
            continue

    moveSearchResults()        

def moveSearchResults(debug = True):    
    modVal   = 500

    files    = findExt(getSearchArtistsDir(), ext=".p")
    for ifile in files:
        discID   = getBaseFilename(ifile)
        modValue = getDiscIDHashMod(discID, modval=modVal)
        outfile  = setSubFile(getArtistsDir(), str(modValue), discID+".p")
        moveFile(ifile, outfile, forceMove = True, debug = True)

    files    = findExt(getSearchArtistsDBDir(), ext=".p")
    for ifile in files:
        removeFile(ifile, debug = True)

        